In [1]:
# !pip install selenium
# !pip install django
# !pip install django-embed-video

In [1]:
import csv
import uuid
from selenium import webdriver
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd

import requests
import urllib
import os

import numpy as np

In [3]:
# ----------------- Crawl data -----------------
titles  = []
years   = []
genres  = []
plots   = []
posters = []
ratings = []
trailors= []

data=[]
link_films = []
# ---------------------------------------------------

In [ ]:
s=Service('C:/Users/Van Thong/Downloads/chromedriver.exe')
driver = webdriver.Chrome(service=s) 
for i in range(1, 501):
    driver.get('https://flixgo.up.railway.app/movies-catalog/')
    your_input = driver.find_element(By.XPATH,'//div//ul/form[@method="post"]//input[@id="page"]')
    your_input.clear()
    your_input.send_keys(str(i))
    your_input.send_keys(Keys.ENTER)
    link_film=driver.find_elements(By.XPATH,"//div//a[@class='card__play']")
    for link in link_film:
        link = link.get_attribute("href")
        link_films.append({"link":link})
driver.quit()
print(link_films)

In [ ]:
for i in link_films:
    driver = webdriver.Chrome(service=s)
    driver.get(i["link"])
    
    driver.execute_script("document.querySelectorAll('body > section.section.details > div:nth-child(3) > div.row > div:nth-child(2) > div > div > div.col-12.col-sm-8.col-md-8.col-lg-9.col-xl-7 > div > ul > li:nth-child(1) > span').forEach(el => el.remove());");

    driver.execute_script("document.querySelectorAll('body > section.section.details > div:nth-child(3) > div.row > div:nth-child(2) > div > div > div.col-12.col-sm-8.col-md-8.col-lg-9.col-xl-7 > div > ul > li:nth-child(2) > span').forEach(el => el.remove());");

    # Lấy tên phim
    title = driver.find_element(By.XPATH,"//h1[@class='details__title']").text
   
    # ------------------Lấy năm phát hành------------------
    try:
        year = driver.find_element(By.XPATH,"/html/body/section[1]/div[3]/div[1]/div[2]/div/div/div[2]/div/ul/li[2]").text
        # Chỉ lấy mỗi năm
        year = year.split("-")[0]
    except:
        year = ''
    years.append({'year': year})
    # ------------------------------------------------------

    # ---------------------Lấy thể loại phim---------------------
    genre = driver.find_elements(By.XPATH,"/html/body/section[1]/div[3]/div[1]/div[2]/div/div/div[2]/div/ul/li/a")
    temp_genres={'genre':[]}

    for i in genre:
        temp_genres['genre'].append(i.text)
    genres_str = ', '.join(temp_genres['genre'])
    new_dict = {'genre': genres_str}
    genres.append(new_dict)
    # --------------------------------------------------------------

    # Lấy Overview
    try:
        plot = driver.find_element(By.XPATH,"//div[@class='card__description card__description--details']").text
    except:
        plot = ''
    # Lấy đánh giá
    try:
        rating = driver.find_element(By.XPATH,"//span[@class='card__rate']").text
    except:
        rating = ''
    # Lấy link poster 
    driver.execute_script("window.scrollTo(0, 990000)")
    poster = driver.find_element(By.XPATH,"//div[@class='card__cover']//img").get_attribute("src")
    
    # Lấy link trailer link youtobe
    # Nếu trailer không có thì sẽ trả về rổng
    try:
        trailor = driver.find_element(By.XPATH,"//div[@class='col-12 col-xl-6']//iframe").get_attribute("src")
    except:
        trailor = None

    # thêm vào list các dict tương ứng
    titles.append({'title': title})
    ratings.append({'rating': rating})
    posters.append({'poster': poster})
    plots.append({'plot': plot})
    trailors.append({'trailor': trailor})

    driver.close()

In [ ]:
# Lưu dữ liệu trên vào file csv
with open('data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['title', 'year', 'genre', 'plot', 'poster', 'rating', 'trailor'])
    for i in range(len(titles)):
        writer.writerow([titles[i]['title'], years[i]['year'], genres[i]['genre'], plots[i]['plot'], posters[i]['poster'], ratings[i]['rating'], trailors[i]['trailor']])


In [3]:
# Đọc file csv
df = pd.read_csv('data_done_NEW_2.csv', encoding='utf-8')
len(df)

9972

In [4]:
df.head()

,movie_id,title,year,genres,overview,language,poster,rating,trailer,runtime,budget,revenue,directors,director_images,top_cast,top_cast_images
0,502356,The Super Mario Bros. Movie,2023,"Animation,Adventure,Family,Fantasy,Comedy","While working underground to fix a water main,...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.514,https://www.youtube.com/watch?v=LTFGH0rJ-EY,1h 32m,100000000,876438061,"Michael Jelenic,Aaron Horvath",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Chris Pratt,Anya Taylor-Joy,Charlie Day,Jack B...",https://www.themoviedb.org/t/p/w138_and_h175_f...
1,700391,65,2023,"Science Fiction,Adventure,Thriller,Action","65 million years ago, the only 2 survivors of ...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.311,https://www.youtube.com/watch?v=2kgJuEX4yhc,1h 33m,45000000,55000000,"Scott Beck,Bryan Woods",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Driver,Ariana Greenblatt,Chloe Coleman,Ni...",https://www.themoviedb.org/t/p/w138_and_h175_f...
2,315162,Puss in Boots: The Last Wish,2022,"Animation,Family,Fantasy,Adventure,Comedy",Puss in Boots discovers that his passion for a...,en,https://www.themoviedb.org/t/p/w220_and_h330_f...,8.299,https://www.youtube.com/watch?v=z-E5pTQVW8w,1h 43m,90000000,480000000,Joel Crawford,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Antonio Banderas,Salma Hayek,Harvey Guillén,Wa...",https://www.themoviedb.org/t/p/w138_and_h175_f...
3,638974,Murder Mystery 2,2023,"Comedy,Mystery,Action","After starting their own detective agency, Nic...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.549,https://www.youtube.com/watch?v=jeVNn7-FQ64,1h 31m,0,0,Jeremy Garelick,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Sandler,Jennifer Aniston,Mark Strong,Méla...",https://www.themoviedb.org/t/p/w138_and_h175_f...
4,736790,Chupa,2023,"Adventure,Fantasy,Family","While visiting family in Mexico, a lonely boy ...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.520,https://www.youtube.com/watch?v=bJ4ENBghzSs,1h 35m,0,0,Jonás Cuarón,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Demián Bichir,Evan Whitten,Christian Slater,As...",https://www.themoviedb.org/t/p/w138_and_h175_f...


In [5]:
# Xoá cột movie_id
df.drop('movie_id', axis=1, inplace=True)
df.head()

,title,year,genres,overview,language,poster,rating,trailer,runtime,budget,revenue,directors,director_images,top_cast,top_cast_images
0,The Super Mario Bros. Movie,2023,"Animation,Adventure,Family,Fantasy,Comedy","While working underground to fix a water main,...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.514,https://www.youtube.com/watch?v=LTFGH0rJ-EY,1h 32m,100000000,876438061,"Michael Jelenic,Aaron Horvath",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Chris Pratt,Anya Taylor-Joy,Charlie Day,Jack B...",https://www.themoviedb.org/t/p/w138_and_h175_f...
1,65,2023,"Science Fiction,Adventure,Thriller,Action","65 million years ago, the only 2 survivors of ...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.311,https://www.youtube.com/watch?v=2kgJuEX4yhc,1h 33m,45000000,55000000,"Scott Beck,Bryan Woods",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Driver,Ariana Greenblatt,Chloe Coleman,Ni...",https://www.themoviedb.org/t/p/w138_and_h175_f...
2,Puss in Boots: The Last Wish,2022,"Animation,Family,Fantasy,Adventure,Comedy",Puss in Boots discovers that his passion for a...,en,https://www.themoviedb.org/t/p/w220_and_h330_f...,8.299,https://www.youtube.com/watch?v=z-E5pTQVW8w,1h 43m,90000000,480000000,Joel Crawford,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Antonio Banderas,Salma Hayek,Harvey Guillén,Wa...",https://www.themoviedb.org/t/p/w138_and_h175_f...
3,Murder Mystery 2,2023,"Comedy,Mystery,Action","After starting their own detective agency, Nic...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.549,https://www.youtube.com/watch?v=jeVNn7-FQ64,1h 31m,0,0,Jeremy Garelick,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Sandler,Jennifer Aniston,Mark Strong,Méla...",https://www.themoviedb.org/t/p/w138_and_h175_f...
4,Chupa,2023,"Adventure,Fantasy,Family","While visiting family in Mexico, a lonely boy ...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.520,https://www.youtube.com/watch?v=bJ4ENBghzSs,1h 35m,0,0,Jonás Cuarón,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Demián Bichir,Evan Whitten,Christian Slater,As...",https://www.themoviedb.org/t/p/w138_and_h175_f...


In [8]:
# Coi runtimes
df['runtime'].head()

0    1h 32m
1    1h 35m
2    1h 33m
3    1h 43m
4    1h 56m
Name: runtime, dtype: object

In [7]:
# Cover runtimes thành giờ phút
# Ví dụ: runtime = 103 phút => 1h 43m
def convert_runtime(runtime):
    if runtime == 0:
        return ''
    else:
        hours = runtime // 60
        minutes = runtime % 60
        return str(hours) + 'h ' + str(minutes) + 'm'

df['runtime'] = df['runtime'].apply(convert_runtime)
df['runtime'].head()


0    1h 32m
1    1h 35m
2    1h 33m
3    1h 43m
4    1h 56m
Name: runtime, dtype: object

In [11]:
df.head()

,title,year,genres,overview,language,poster,rating,trailer,runtime,budget,revenue,directors,director_images,top_cast,top_cast_images
0,The Super Mario Bros. Movie,2023,"Animation,Adventure,Family,Fantasy,Comedy","While working underground to fix a water main,...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.508,https://www.youtube.com/watch?v=,1h 32m,100000000,876438061,"Michael Jelenic,Aaron Horvath",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Chris Pratt,Anya Taylor-Joy,Charlie Day,Jack B...",https://www.themoviedb.org/t/p/w138_and_h175_f...
1,Chupa,2023,"Adventure,Fantasy,Family","While visiting family in Mexico, a lonely boy ...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.537,https://www.youtube.com/watch?v=,1h 35m,0,0,Jonás Cuarón,Unknown,"Demián Bichir,Evan Whitten,Christian Slater,As...",https://www.themoviedb.org/t/p/w138_and_h175_f...
2,65,2023,"Science Fiction,Adventure,Thriller,Action","65 million years ago, the only 2 survivors of ...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.316,https://www.youtube.com/watch?v=,1h 33m,45000000,55000000,"Scott Beck,Bryan Woods",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Driver,Ariana Greenblatt,Chloe Coleman,Ni...",https://www.themoviedb.org/t/p/w138_and_h175_f...
3,Puss in Boots: The Last Wish,2022,"Animation,Family,Fantasy,Adventure,Comedy",Puss in Boots discovers that his passion for a...,en,https://www.themoviedb.org/t/p/w220_and_h330_f...,8.300,https://www.youtube.com/watch?v=,1h 43m,90000000,480000000,Joel Crawford,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Antonio Banderas,Salma Hayek,Harvey Guillén,Wa...",https://www.themoviedb.org/t/p/w138_and_h175_f...
4,Creed III,2023,"Drama,Action","After dominating the boxing world, Adonis Cree...",en,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.274,https://www.youtube.com/watch?v=,1h 56m,75000000,269000000,Michael B. Jordan,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Michael B. Jordan,Tessa Thompson,Jonathan Majo...",https://www.themoviedb.org/t/p/w138_and_h175_f...


In [6]:
language_dict = {'en': 'English', 'ko': 'Korean', 'es': 'Spanish', 'no': 'Norwegian', 'ja': 'Japanese', 'tr': 'Turkish', 'uk': 'Ukrainian', 'th': 'Thai', 'ru': 'Russian', 'it': 'Italian', 'fi': 'Finnish', 'pl': 'Polish', 'cn': 'Chinese (simplified)', 'de': 'German', 'fr': 'French', 'ro': 'Romanian', 'is': 'Icelandic', 'te': 'Telugu', 'id': 'Indonesian', 'zh': 'Chinese (traditional)', 'da': 'Danish', 'nl': 'Dutch', 'ta': 'Tamil', 'ca': 'Catalan', 'mk': 'Macedonian', 'tl': 'Tagalog', 'pt': 'Portuguese', 'ar': 'Arabic', 'sv': 'Swedish', 'hi': 'Hindi', 'vi': 'Vietnamese', 'gl': 'Galician', 'sr': 'Serbian', 'xx': 'Unknown Language', 'el': 'Greek', 'nb': 'Norwegian Bokmål', 'bn': 'Bengali', 'my': 'Burmese', 'ml': 'Malayalam', 'fa': 'Persian', 'kn': 'Kannada', 'ga': 'Irish', 'ur': 'Urdu', 'km': 'Khmer', 'sh': 'Serbo-Croatian', 'cs': 'Czech', 'la': 'Latin', 'yo': 'Yoruba', 'hu': 'Hungarian', 'lv': 'Latvian'}

df['language'] = df['language'].map(language_dict)

In [7]:
df.head()

,title,year,genres,overview,language,poster,rating,trailer,runtime,budget,revenue,directors,director_images,top_cast,top_cast_images
0,The Super Mario Bros. Movie,2023,"Animation,Adventure,Family,Fantasy,Comedy","While working underground to fix a water main,...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.514,https://www.youtube.com/watch?v=LTFGH0rJ-EY,1h 32m,100000000,876438061,"Michael Jelenic,Aaron Horvath",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Chris Pratt,Anya Taylor-Joy,Charlie Day,Jack B...",https://www.themoviedb.org/t/p/w138_and_h175_f...
1,65,2023,"Science Fiction,Adventure,Thriller,Action","65 million years ago, the only 2 survivors of ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.311,https://www.youtube.com/watch?v=2kgJuEX4yhc,1h 33m,45000000,55000000,"Scott Beck,Bryan Woods",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Driver,Ariana Greenblatt,Chloe Coleman,Ni...",https://www.themoviedb.org/t/p/w138_and_h175_f...
2,Puss in Boots: The Last Wish,2022,"Animation,Family,Fantasy,Adventure,Comedy",Puss in Boots discovers that his passion for a...,English,https://www.themoviedb.org/t/p/w220_and_h330_f...,8.299,https://www.youtube.com/watch?v=z-E5pTQVW8w,1h 43m,90000000,480000000,Joel Crawford,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Antonio Banderas,Salma Hayek,Harvey Guillén,Wa...",https://www.themoviedb.org/t/p/w138_and_h175_f...
3,Murder Mystery 2,2023,"Comedy,Mystery,Action","After starting their own detective agency, Nic...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.549,https://www.youtube.com/watch?v=jeVNn7-FQ64,1h 31m,0,0,Jeremy Garelick,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Sandler,Jennifer Aniston,Mark Strong,Méla...",https://www.themoviedb.org/t/p/w138_and_h175_f...
4,Chupa,2023,"Adventure,Fantasy,Family","While visiting family in Mexico, a lonely boy ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.520,https://www.youtube.com/watch?v=bJ4ENBghzSs,1h 35m,0,0,Jonás Cuarón,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Demián Bichir,Evan Whitten,Christian Slater,As...",https://www.themoviedb.org/t/p/w138_and_h175_f...


In [19]:
# Lưu dữ liệu vào file csv
df.to_csv('output_MAIN.csv', index=False)

In [16]:
# liệt kê các giá trị trong cột language
df['language'].unique()


array(['en', 'ko', 'es', 'no', 'ja', 'tr', 'uk', 'th', 'ru', 'it', 'fi',
       'pl', 'cn', 'de', 'fr', 'ro', 'is', 'te', 'id', 'zh', 'da', 'nl',
       'ta', 'ca', 'mk', 'tl', 'pt', 'ar', 'sv', 'hi', 'vi', 'gl', 'sr',
       'xx', 'el', 'nb', 'bn', 'my', 'ml', 'fa', 'kn', 'ga', 'ur', 'km',
       'sh', 'cs', 'la', 'yo', 'hu', 'lv'], dtype=object)

`Chuyển đổi kiểu dữ liệu của cột 'year' sang kiểu số nguyên (integer)`

In [29]:
df = df.dropna(subset=['year'])
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['year'])

df['year'] = df['year'].astype(int) 

`Kiểm tra df có giá trị trùng không xóa các dữ liệu trùng đó`


In [4]:
df.duplicated(subset=['title', 'year']).sum()
df = df.drop_duplicates(subset=['title', 'year'], keep='first')
len(df)

8950

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('film_DONE_MAIN.csv', encoding='utf-8')
len(df)

10000

In [4]:
df1 = pd.read_csv('directors_cast_MAIN.csv', encoding='utf-8')
len(df1)

9996

In [17]:
df1.head()

,movie_id,directors,director_images,top_cast,top_cast_images
0,movie,"Michael Jelenic,Aaron Horvath",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Chris Pratt,Anya Taylor-Joy,Charlie Day,Jack B...",https://www.themoviedb.org/t/p/w138_and_h175_f...
1,movie,Chad Stahelski,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Keanu Reeves,Donnie Yen,Bill Skarsgård,Ian McS...",https://www.themoviedb.org/t/p/w138_and_h175_f...
2,movie,David F. Sandberg,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Zachary Levi,Asher Angel,Jack Dylan Grazer,Ada...",https://www.themoviedb.org/t/p/w138_and_h175_f...
3,movie,"Scott Beck,Bryan Woods",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Driver,Ariana Greenblatt,Chloe Coleman,Ni...",https://www.themoviedb.org/t/p/w138_and_h175_f...
4,movie,Joel Crawford,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Antonio Banderas,Salma Hayek,Harvey Guillén,Wa...",https://www.themoviedb.org/t/p/w138_and_h175_f...


In [18]:
# Xem 5 giá trị cuối df
df.head()

,Title,Year,Genre,Overview,Language,Poster,Rating,Trailer,Runtime,Budget,Revenue
0,The Super Mario Bros. Movie,05/04/2023,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...",English,https://image.tmdb.org/t/p/w500/qNBAXBIQlnOThr...,7.500,https://www.youtube.com/watch?v=LTFGH0rJ-EY,1h 43m,"100,000,000.00","724,289,031.00"
1,Shazam! Fury of the Gods,15/03/2023,"Action, Comedy, Fantasy","Billy Batson and his foster siblings, who tran...",English,https://image.tmdb.org/t/p/w500/3GrRgt6CiLIUXU...,6.914,https://www.youtube.com/watch?v=gardMnJszqc,1h 32m,"125,000,000.00",0
2,Avatar: The Way of Water,14/12/2022,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,English,https://image.tmdb.org/t/p/w500/t6HIqrRAclMCA6...,7.725,https://www.youtube.com/watch?v=Sw9hdSjzZjc,2h 10m,"350,000,000.00","2,312,335,665.00"
3,Creed III,01/03/2023,"Drama, Action","After dominating the boxing world, Adonis Cree...",English,https://image.tmdb.org/t/p/w500/cvsXj3I9Q2iyyI...,7.269,https://www.youtube.com/watch?v=qphf_-PPaaI,1h 56m,"75,000,000.00","269,000,000.00"
4,65,02/03/2023,"Science Fiction, Adventure, Thriller, Action","65 million years ago, the only 2 survivors of ...",English,https://image.tmdb.org/t/p/w500/rzRb63TldOKdKy...,6.353,https://www.youtube.com/watch?v=2kgJuEX4yhc,1h 31m,"45,000,000.00","55,000,000.00"


In [40]:
df = pd.read_csv('DataBase.csv', encoding='utf-8')
df

,title,year,genres,overview,language,poster,rating,trailer,runtime,budget,revenue,directors,director_images,top_cast,top_cast_images
0,The Super Mario Bros. Movie,2023,"Animation,Adventure,Family,Fantasy,Comedy","While working underground to fix a water main,...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.508,https://www.youtube.com/watch?v=LTFGH0rJ-EY,1h 32m,"100,000,000.0","876,438,061.0","Michael Jelenic,Aaron Horvath",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Chris Pratt,Anya Taylor-Joy,Charlie Day,Jack B...",https://www.themoviedb.org/t/p/w138_and_h175_f...
1,Chupa,2023,"Adventure,Fantasy,Family","While visiting family in Mexico, a lonely boy ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.537,https://www.youtube.com/watch?v=bJ4ENBghzSs,1h 35m,NaN,NaN,Jonás Cuarón,Unknown,"Demián Bichir,Evan Whitten,Christian Slater,As...",https://www.themoviedb.org/t/p/w138_and_h175_f...
2,65,2023,"Science Fiction,Adventure,Thriller,Action","65 million years ago, the only 2 survivors of ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.316,https://www.youtube.com/watch?v=POfC3TrDO24,1h 33m,"45,000,000.0","55,000,000.0","Scott Beck,Bryan Woods",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Driver,Ariana Greenblatt,Chloe Coleman,Ni...",https://www.themoviedb.org/t/p/w138_and_h175_f...
3,Puss in Boots: The Last Wish,2022,"Animation,Family,Fantasy,Adventure,Comedy",Puss in Boots discovers that his passion for a...,English,https://www.themoviedb.org/t/p/w220_and_h330_f...,8.300,https://www.youtube.com/watch?v=gardMnJszqc,1h 43m,"90,000,000.0","480,000,000.0",Joel Crawford,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Antonio Banderas,Salma Hayek,Harvey Guillén,Wa...",https://www.themoviedb.org/t/p/w138_and_h175_f...
4,Creed III,2023,"Drama,Action","After dominating the boxing world, Adonis Cree...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.274,https://www.youtube.com/watch?v=2kgJuEX4yhc,1h 56m,"75,000,000.0","269,000,000.0",Michael B. Jordan,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Michael B. Jordan,Tessa Thompson,Jonathan Majo...",https://www.themoviedb.org/t/p/w138_and_h175_f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9970,Jessica Darling's It List,2016,Family,"Jessica Darling – a smart, witty, opinionated ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,5.921,https://www.youtube.com/watch?v=bNruMmoqdO8,1h 19m,NaN,NaN,Ali Scher,Unknown,"Chloe East,Emma Rayne Lyle,Ashley Liao,Eva Bel...",https://www.themoviedb.org/t/p/w138_and_h175_f...
9971,ゴジラ・ミニラ・ガバラ オール怪獣大進撃,1969,"Fantasy,Science Fiction,Family",Ichiro Miki is a child living in the industria...,Japanese,https://www.themoviedb.org/t/p/w220_and_h330_f...,4.996,https://www.youtube.com/watch?v=U7fkvIxrEoE,1h 10m,NaN,"2,399,280.0",Ishirō Honda,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Tomonori Yazaki,Hideyo Amamoto,Sachio Sakai,Ka...",https://www.themoviedb.org/t/p/w138_and_h175_f...
9972,劇場版 魔法少女まどか☆マギカ [前編] 始まりの物語,2012,"Thriller,Animation,Fantasy","Madoka Kaname, an ordinary middle-schooler, al...",Japanese,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.553,https://www.youtube.com/watch?v=ZIzkq6vau6Y,2h 11m,NaN,NaN,Yukihiro Miyamoto,Unknown,"Aoi Yuki,Chiwa Saito,Eri Kitamura,Kaori Mizuha...",https://www.themoviedb.org/t/p/w138_and_h175_f...
9973,Target Number One,2020,"Thriller,Crime","Ex heroin junkie, Daniel Léger, gets involved ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.330,https://www.youtube.com/watch?v=ljN7SEEwhmM,2h 15m,"138,000.0",NaN,Daniel Roby,Unknown,"Antoine Olivier Pilon,Josh Hartnett,Stephen Mc...",https://www.themoviedb.org/t/p/w138_and_h175_f...


In [8]:
# cover sang tiền đô budget revenue
# Ví dụ: 100000000.0 --> 100,000,000.00
def convert_money(money):
    if money == 0:
        return ''
    else:
        return "{:,}".format(money)

df['budget'] = df['budget'].apply(convert_money)
df['revenue'] = df['revenue'].apply(convert_money)
    


In [9]:
df.head()

,title,year,genres,overview,language,poster,rating,trailer,runtime,budget,revenue,directors,director_images,top_cast,top_cast_images
0,The Super Mario Bros. Movie,2023,"Animation,Adventure,Family,Fantasy,Comedy","While working underground to fix a water main,...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,7.514,https://www.youtube.com/watch?v=LTFGH0rJ-EY,1h 32m,"100,000,000","876,438,061","Michael Jelenic,Aaron Horvath",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Chris Pratt,Anya Taylor-Joy,Charlie Day,Jack B...",https://www.themoviedb.org/t/p/w138_and_h175_f...
1,65,2023,"Science Fiction,Adventure,Thriller,Action","65 million years ago, the only 2 survivors of ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.311,https://www.youtube.com/watch?v=2kgJuEX4yhc,1h 33m,"45,000,000","55,000,000","Scott Beck,Bryan Woods",https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Driver,Ariana Greenblatt,Chloe Coleman,Ni...",https://www.themoviedb.org/t/p/w138_and_h175_f...
2,Puss in Boots: The Last Wish,2022,"Animation,Family,Fantasy,Adventure,Comedy",Puss in Boots discovers that his passion for a...,English,https://www.themoviedb.org/t/p/w220_and_h330_f...,8.299,https://www.youtube.com/watch?v=z-E5pTQVW8w,1h 43m,"90,000,000","480,000,000",Joel Crawford,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Antonio Banderas,Salma Hayek,Harvey Guillén,Wa...",https://www.themoviedb.org/t/p/w138_and_h175_f...
3,Murder Mystery 2,2023,"Comedy,Mystery,Action","After starting their own detective agency, Nic...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.549,https://www.youtube.com/watch?v=jeVNn7-FQ64,1h 31m,,,Jeremy Garelick,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Adam Sandler,Jennifer Aniston,Mark Strong,Méla...",https://www.themoviedb.org/t/p/w138_and_h175_f...
4,Chupa,2023,"Adventure,Fantasy,Family","While visiting family in Mexico, a lonely boy ...",English,https://www.themoviedb.org/t/p/w220_and_h330_f...,6.520,https://www.youtube.com/watch?v=bJ4ENBghzSs,1h 35m,,,Jonás Cuarón,https://www.themoviedb.org/t/p/w138_and_h175_f...,"Demián Bichir,Evan Whitten,Christian Slater,As...",https://www.themoviedb.org/t/p/w138_and_h175_f...


In [10]:
# Lưu dữ liệu vào file csv
df.to_csv('DataBase_1.csv', index=False)


In [39]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'